# Dataset

In [8]:
from sklearn.datasets import load_iris

# Carregando dataset de iris
dataset = load_iris()

# Guardando dados de entrada e as saídas desejadas para cada amostra de treinamento
amostras = dataset.data[:130]
y_desejado = dataset.target[:130]

# Parâmetros da rede neural

In [9]:
import numpy as np
import matplotlib.pyplot as plt
import pyswarms as ps

# Arquiteura da rede neural
n_entrada = 4                # número de entradas na camada de entrada
n_oculta = 20                # número de neurônios na camada oculta
n_saida = 3                  # número de classes na camada de saídas
n_amostras = len(amostras)   # número de amostras

# Ativação, propagação e compração dos resultados

In [10]:
def ativacao1(p):
    # Construindo vetores iniciais de pesos e bias a partir da lista de parâmetros
    W1 = p[0:80].reshape((n_entrada,n_oculta))      # pesos oculta-entrada
    b1 = p[80:100].reshape((n_oculta,))             # bias oculta-entrada
    W2 = p[100:160].reshape((n_oculta,n_saida))     # pesos saída-oculta
    b2 = p[160:163].reshape((n_saida,))             # bias saída-oculta

    # Ativando camada oculta pela função de ativação sigmoid
    z1 = amostras.dot(W1) + b1      # nets da camada oculta
    a1 = np.tanh(z1)                # ativação da camada oculta
    y_obtido = a1.dot(W2) + b2      # nets da camada de saída
    return y_obtido                 # saídas da rede neural

# Propagação direta da rede neural
def propagacao(particula):
    y_obtido = ativacao1(particula)

    # Ativação da camada de saída pela função de softmax
    exp_scores = np.exp(y_obtido)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Calcula o erro da saída de rede
    corect_logprobs = -np.log(probs[range(n_amostras), y_desejado]) # erro por saída
    erro = np.sum(corect_logprobs) / n_amostras                     # erro médio total

    return erro

# Calculando a acurácia
def avaliacao(posicao):
    y_obtido = ativacao1(posicao)
    resultado = np.argmax(y_obtido, axis=1)
    return resultado

# Execução do PSO para otimização

In [4]:
import pso

def funcao_obj(enxame):
    n_particulas = enxame.shape[0]
    enxame = [propagacao(enxame[i]) for i in range(n_particulas)]
    return np.array(enxame)

"""otimizacao = pso.Pso(100, 100)
lb = np.array([-1] * dimensoes)
ub = np.array([1] * dimensoes)
posicao, custo = otimizacao.run(avaliacao, lb, ub)"""

# Definindo fator de aceleração 1 e 2 e peso de inércia para o PSO
parametrosPSO = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
dimensoes = (n_entrada * n_oculta) + (n_oculta * n_saida) + n_oculta + n_saida
n_particulas = 100

# Iniciando PSO
otimizacao = ps.single.GlobalBestPSO(n_particles=n_particulas, dimensions=dimensoes, options=parametrosPSO)

# Realizando treinamento/otimização
custo, posicao = otimizacao.optimize(funcao_obj, iters=1000, verbose=False)

# Resultado do treinamento

In [5]:
print("Valor ótimo de minização: ", custo)
print("Pesos otimizados da camada oculta:\n", posicao[:80])
print("Bias otimizados da camada oculta:\n", posicao[80:100])
print("Pesos otimizados da camada de saída:\n", posicao[100:160])
print("Bias otimizados da camada de saída:\n", posicao[160:])

Valor ótimo de minização:  0.00023724544633023484
Pesos otimizados da camada oculta:
 [ 1.45907353  1.1921351   0.89154503  1.03765813 -0.82479544 -0.77148451
  1.42544754  1.56380511 -1.45457078  0.22338136  0.91026312 -1.75123616
 -0.59340242  1.73791052  4.257714    1.47377241  0.95535156 -0.31947122
  1.49960368 -1.22085904  0.46564033 -3.10475643 -0.48281301  2.35709015
 -0.37575626 -0.86994926  1.29075609 -0.91871195 -1.09453694  1.69849375
 -1.02874338 -1.43076525 -1.98436526 -3.91926216  0.31973664 -0.56898465
  0.53808463  0.70756668  0.38862301  0.49300825  0.07168587  0.42674066
  0.51635703 -3.1419513  -0.97211482 -1.36690246  0.91881042 -0.59847132
  0.25704509 -1.96302628  0.65843132 -0.18420562  2.42287038  0.5353109
 -1.22591361  2.84039216 -3.83692799  1.24790098 -1.93347273  0.71714397
 -0.28800575  1.41340364  1.04720025 -0.17879224 -3.84239269  1.06488206
  0.81767592  1.42301319  0.08406273 -1.15458943 -1.53956771 -6.15479647
  3.0050031   0.62389869 -4.20336026 -2

# Avaliação do treinamento e do teste

In [6]:
# Acurácia do treino
print("Acurácia de treino: ", (np.mean(avaliacao(posicao) == y_desejado) * 100))

Acurácia de treino:  100.0


# Execução e avaliação do teste

In [7]:
# Guardando dados de entrada e as saídas desejadas para cada amostra de teste
amostras = dataset.data[130:]
y_desejado = dataset.target[130:]

# Acurácia do teste
print("Acurácia de teste: ", (np.mean(avaliacao(posicao) == y_desejado) * 100))

Acurácia de teste:  95.0
